In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv')
df.head()

,User,Track,Rating
0,1,0,27
1,4,0,10
2,8,0,30
3,10,0,30
4,11,0,14


In [4]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = df.copy()
y = df['User']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [5]:
# Importing tensorflow
import tensorflow as tf
# Importing some more libraries
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [6]:
num_items = df.Track.nunique()
num_users = df.User.nunique()

print("USERS: {} ITEMS: {}".format(num_users, num_items))

USERS: 47856 ITEMS: 184


In [7]:
# Normalize in [0, 1]
from sklearn import preprocessing

def normalize(df):
    r = df['Rating'].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(r.reshape(-1,1))
    df_normalized = pd.DataFrame(x_scaled)
    df['Rating'] = df_normalized

normalize(df)

In [8]:
matrix = df.pivot(index='User', columns='Track', values='Rating')
matrix.fillna(0, inplace=True)

In [9]:
users = matrix.index.tolist()
items = matrix.columns.tolist()

matrix = matrix.as_matrix()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [10]:
num_input = num_items
num_hidden_1 = 100
num_hidden_2 = 50

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

In [11]:
# Building the encoder

def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2


# Building the decoder

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2


# Construct model

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)


# Prediction

y_pred = decoder_op


# Targets are the input data.

y_true = X

In [12]:
from keras import losses
from sklearn import metrics
# Define loss and optimizer, minimize the squared errorn
loss = losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)

predictions = pd.DataFrame()

# Define evaluation metrics

eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
#pre, pre_op = metrics.precision(labels=eval_x, predictions=eval_y)

Using TensorFlow backend.


In [13]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()

In [14]:
with tf.Session() as session:
    epochs = 5
    batch_size = 2

    session.run(init)
    session.run(local_init)

    num_batches = int(len(matrix[0]) / batch_size)
    matrix = np.array_split(matrix, num_batches)

    for i in range(epochs):

        avg_cost = 0

        for batch in matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += 1

        avg_cost /= num_batches

        print("Epoch: {} Loss: {}".format(i + 1, avg_cost))

    print("Predictions...")

    matrix = np.concatenate(matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: matrix})

    predictions = predictions.append(pd.DataFrame(preds))

    predictions = predictions.stack().reset_index(name='Rating')
    predictions.columns = ['User', 'Track', 'Rating']
    predictions['User'] = predictions['User'].map(lambda value: users[value])
    predictions['Track'] = predictions['Track'].map(lambda value: items[value])

Epoch: 1 Loss: 1.0
Epoch: 2 Loss: 1.0
Epoch: 3 Loss: 1.0
Epoch: 4 Loss: 1.0
Epoch: 5 Loss: 1.0
Predictions...


In [15]:
print("Filtering out items in training set")

keys = ['User', 'Track']
i1 = predictions.set_index(keys).index
i2 = df.set_index(keys).index

recs = predictions[~i1.isin(i2)]
recs = recs.sort_values(['User', 'Rating'], ascending=[True, False])

Filtering out items in training set


In [16]:
recs.head(5)

,User,Track,Rating
177,0,177,0.746890
179,0,179,0.018403
182,0,182,0.008923
180,0,180,0.007966
169,0,169,0.007734


In [17]:
recs.shape

(8654505, 3)

In [20]:
recs['User'].isin(X_test['User'])

177        False
179        False
182        False
180        False
169        False
164        False
160        False
150        False
139        False
11         False
168        False
149        False
142        False
155        False
173        False
178        False
166        False
156        False
112        False
124        False
165        False
159        False
126        False
167        False
96         False
93         False
138        False
106        False
162        False
73         False
           ...  
8805437     True
8805403     True
8805461     True
8805329     True
8805341     True
8805454     True
8805447     True
8805344     True
8805400     True
8805442     True
8805431     True
8805440     True
8805323     True
8805495     True
8805378     True
8805472     True
8805422     True
8805356     True
8805361     True
8805328     True
8805334     True
8805363     True
8805463     True
8805358     True
8805412     True
8805433     True
8805425     True
8805492     Tr

In [18]:
#import metrics library
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

X_test['Rating']=(X_test['Rating']-X_test['Rating'].min())/(X_test['Rating'].max()-X_test['Rating'].min())

y_pred = pd.merge(recs, X_test, how='inner', on=['User', 'Track'])

print(X_test.shape)
print(y_test.shape)
print(y_pred.shape)

(37750, 3)
(37750,)
(0, 4)


In [ ]:
#print result of rmse
print(float(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))